# ML Alogorithm Convergence Plots
___

I'm making some code to plot and format the convergence data from Chris

# Import Modules

In [1]:
%%capture
%load_ext autoreload
%autoreload 2

In [2]:
# Setting Custom Paths ********************************************************
# *****************************************************************************
import os
import sys

sys.path.insert(0, os.path.join(
    os.environ["PROJ_irox"],
    "data"))

# Python Modules **************************************************************
# *****************************************************************************
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.io as pio

import pandas as pd

# My Modules ******************************************************************
# *****************************************************************************

# Local Imports ***************************************************************
# *****************************************************************************
from parse_plot_data import get_plot_data
from methods import process_data_for_plot, get_layout

from proj_data_irox import (
    proj_dir_name,
    )

# Script Inputs

In [3]:
save_plot = True

zoom_in_ranges = {
    "x": [+00.0, +20.0],
    "y": [-00.8, -00.3],
    }

show_error_bars = True
filled_error_traces = False

# Load and Process Data

## Load Data

In [4]:
df = get_plot_data()

# Processing Data

## Reorganizing last gen. data to match 1st gen structure ordering

In [5]:
# Generation to be the baseline for x-axis ordering
# This gen will have a continious color scale gradient
gen_ordered = "round_1"

if gen_ordered == "round_2":
    gen_other = "round_1"
elif gen_ordered == "round_1":
    gen_other = "round_2"

num_i = df["iro3"][gen_ordered]["ind"].astype(float)
den_i = float(df["iro3"][gen_ordered]["ind"].max())
new_col = list(num_i / den_i)

df["iro3"][gen_ordered, "ordering_color_grad"] = new_col
df["iro3"].sort_index(axis=1, inplace=True)



id_color_grad_mapping = dict(zip(
    df["iro3"][gen_ordered]["id"],
    df["iro3"][gen_ordered]["ordering_color_grad"],
    ))


def struct_mapping(row_i):
    """
    """
    order_i = id_color_grad_mapping[row_i["id"]]

    return(order_i)

df["iro3"][gen_other, "ordering_color_grad"] = df["iro3"][gen_other].apply(
    struct_mapping,
    axis=1,
    )

df["iro3"].sort_index(axis=1, inplace=True)

## Processing data into plotting traces

In [6]:
color_i = "rgba(152,152,152,0.9)"

data_round_2 = process_data_for_plot(
    df["iro3"]["round_2"],
    color0=color_i,
    color1=color_i,
    name="round2",
    marker_color_mode="array",  # single, array
    energy_key="form_e_0",
    marker_color_key="ordering_color_grad",
    uncertainty_key="uncert_0",
    process_text_for_hover=True,
    explicit_error_bars=show_error_bars,
    filled_error_traces=filled_error_traces,
    )

data_round_1 = process_data_for_plot(
    df["iro3"]["round_1"],
    color0=color_i,
    color1=color_i,
    name="round1",
    marker_color_mode="array",  # single, array
    energy_key="form_e_0",
    marker_color_key="ordering_color_grad",
    uncertainty_key="uncert_0",
    process_text_for_hover=True,
    explicit_error_bars=show_error_bars,
    filled_error_traces=filled_error_traces,
    )

# Plotting Figures

## Plot Layout Config

In [7]:
layout = get_layout()

layout["yaxis"]["range"] = [-1., 2.]
layout["yaxis"]["dtick"] = 0.5
layout["yaxis"]["title"] = ""

layout["xaxis"]["title"] = ""
layout["xaxis"]["range"] = [0, 275]
layout["xaxis"]["showticklabels"] = False

layout["width"] = 1.05 * 18.7 * 37.795275591
layout["height"] = 0.4 * 18.7 * 37.795275591
layout["showlegend"] = False


layout["shapes"] = [
    {
        "type": "rect",
        "xref": "x",
        "yref": "y",
        "x0": zoom_in_ranges["x"][0],
        "y0": zoom_in_ranges["y"][0],
        "x1": zoom_in_ranges["x"][1],
        "y1": zoom_in_ranges["y"][1],
        "line": {
            "width": 0.25,
            "color": "black",
            },
        },
    ]

In [8]:
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"

## Generation 0

In [9]:
fig = dict(data=data_round_1, layout=layout)
pio.write_image(fig, 'images/iro3_ml_convergence_gen_0.svg')
py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_convergence",
        "iro3",
        "iro3_ml_convergence_gen_0",
        ),
    )

/home/raulf2012/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## Generation 1

In [10]:
fig = dict(data=data_round_2, layout=layout)
pio.write_image(fig, 'images/iro3_ml_convergence_gen_1.svg')
py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_convergence",
        "iro3",
        "iro3_ml_convergence_gen_1",
        ),
    )

# Zoomed in Plots

In [11]:
marker_size = 10

In [12]:
layout["xaxis"]["range"] = [0, 20]
layout["yaxis"]["range"] = [-0.8, -0.3]

layout["yaxis"]["dtick"] = 0.2
layout["width"] = (2. * 18.7 * 37.795275591) / 4.5
layout["height"] = (1. * 18.7 * 37.795275591) / 4.0

# Removing shapes
layout.pop("shapes")

[{'type': 'rect',
  'xref': 'x',
  'yref': 'y',
  'x0': 0.0,
  'y0': -0.8,
  'x1': 20.0,
  'y1': -0.3,
  'line': {'width': 0.25, 'color': 'black'}}]

In [13]:
for i in data_round_1:
    i_dict = i.to_plotly_json()
    if "name" in i_dict.keys():
        if "energies" in i_dict["name"]:
            trace_j = i
            trace_j["marker"]["size"] = marker_size

fig = dict(data=data_round_1, layout=layout)
pio.write_image(fig, 'images/iro3_ml_convergence_gen_0__zoomed_in.svg')
py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_convergence",
        "iro3",
        "iro3_ml_convergence_gen_0__zoomed_in",
        ),
    )

In [14]:
for i in data_round_2:
    i_dict = i.to_plotly_json()
    if "name" in i_dict.keys():
        if "energies" in i_dict["name"]:
            trace_j = i
            trace_j["marker"]["size"] = marker_size

fig = dict(data=data_round_2, layout=layout)
pio.write_image(fig, 'images/iro3_ml_convergence_gen_1__zoomed_in.svg')
py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_convergence",
        "iro3",
        "iro3_ml_convergence_gen_1__zoomed_in",
        ),
    )

# end_of_file

In [15]:
# struct_order_mapping = dict(zip(
#     df["iro3"]["round_2"]["id"].tolist(),
#     df["iro3"]["round_2"]["ind"].tolist(),
#     ))

# df_iro3_r1 = df["iro3"]["round_1"]

# def struct_mapping(row_i):
#     """
#     """
#     order_i = struct_order_mapping[row_i["id"]]

#     return(order_i)

# df_iro3_r1["ordering_column"] = df_iro3_r1.apply(
#     struct_mapping,
#     axis=1,
#     )

# df_iro3_r1_sorted = df_iro3_r1.sort_values("ordering_column")
# df_iro3_r1_sorted["ind_old"] = df_iro3_r1_sorted["ind"]
# df_iro3_r1_sorted["ind"] = df_iro3_r1_sorted["ordering_column"]

# data_final = []

# data1 = process_data_for_plot(
#     # df["iro3"]["round_1"],
#     df_iro3_r1_sorted,
#     color0="rgba(152,152,152,1.0)",
#     color1="rgba(152,152,152,1.0)",
#     # color1="blue",
#     name="round1",
#     explicit_error_bars=show_error_bars,
#     )
# data_final += data1

# data2 = process_data_for_plot(
#     df["iro3"]["round_2"],
#     # df_iro3_r2_sorted,
#     color0="rgba(215,142,82,0.8)",
#     color1="rgba(215,142,82,0.8)",
#     # color1="pink",
#     name="round2",
#     explicit_error_bars=show_error_bars,
#     )
# data_final += data2

# data_final = []

# data1 = process_data_for_plot(
#     df["iro2"]["round_1"],
#     color0="rgba(152,152,152,1.0)",
#     name="round1",
#     )
# data_final += data1

# data2 = process_data_for_plot(
#     df["iro2"]["round_2"],
#     color0="rgba(215,142,82,0.8)",
#     name="round2",
#     )
# data_final += data2

# # data3 = process_data_for_plot(
# #     df["round_3"],
# #     color0="rgba(233,20,140,0.1)",
# #     name="round3",
# #     )
# # data_final += data3

# data_final = []

# data_round_2 = process_data_for_plot(
#     df["iro3"]["round_2"],
#     color0="rgba(215,142,82,0.8)",
#     color1="rgba(215,142,82,0.8)",

#     name="round2",
#     marker_color_mode="array",  # single, array

#     energy_key="form_e_0",
#     marker_color_key="ordering_color_grad",

#     uncertainty_key="uncert_0",
#     )
# data_final += data_round_2


# data_round_1 = process_data_for_plot(
#     df["iro3"]["round_1"],
#     color0="rgba(152,152,152,0.3)",
#     color1="rgba(152,152,152,0.3)",
#     name="round1",
#     marker_color_mode="array",  # single, array
#     energy_key="form_e_0",
#     marker_color_key="ordering_color_grad",

#     uncertainty_key="uncert_0",
#     )
# data_final += data_round_1

# layout = get_layout()

# # layout["xaxis"]["range"]
# layout["yaxis"]["range"] = [-1.5, 2.]

# layout["width"] = 2. * 18.7 * 37.795275591
# layout["height"] = 1. * 18.7 * 37.795275591

# layout["showlegend"] = True
# fig = dict(data=data_final, layout=layout)
# py.iplot(fig,
#     filename=os.path.join(
#         save_dir,
#         "ml_convergence",
#         "iro3_ml_convergence__large_tmp",
#         ),
#     )

# fig = dict(data=data_final, layout=layout)
# py.iplot(fig,
#     filename=os.path.join(
#         save_dir,
#         "ml_convergence",
#         "iro2_ml_convergence__large",
#         ),
#     )

# layout["width"] = 11. * 37.795275591
# layout["height"] = 11. * 37.795275591

# # fig = dict(data=data_final, layout=layout)
# # py.iplot(fig,
# #     filename=os.path.join(
# #         save_dir,
# #         "ml_convergence",
# #         "iro2_ml_convergence__main",
# #         ),
# #     )

# layout = get_layout()

# layout["xaxis"]["range"]
# layout["yaxis"]["range"] = [-1.5, 2.]

# layout["showlegend"] = True
# fig = dict(data=data_final, layout=layout)
# py.iplot(fig,
#     filename=os.path.join(
#         save_dir,
#         "ml_convergence",
#         "iro3_ml_convergence__large",
#         ),
#     )

# layout["width"] = 11. * 37.795275591
# layout["height"] = 11. * 37.795275591
# layout["showlegend"] = False

# # fig = dict(data=data_final, layout=layout)
# # py.iplot(fig,
# #     filename=os.path.join(
# #         save_dir,
# #         "ml_convergence",
# #         "iro3_ml_convergence__main",
# #         ),
# #     )